In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

In [2]:
adata = sc.read_h5ad('./data/Tcell_PDL1.h5ad')

In [3]:
np.unique(adata.obs['Sub_Cluster'])

array(['t_CD4-CXCL13', 't_CD4_Tcm-LMNA', 't_CD4_Tn-LEF1',
       't_CD4_Treg-FOXP3', 't_CD8-CXCL13', 't_CD8_MAIT-KLRB1',
       't_CD8_Teff-GNLY', 't_CD8_Tem-GZMK', 't_CD8_Trm-ZNF683',
       't_Tact-IFI6', 't_Tact-XIST', 't_Tprf-MKI67'], dtype=object)

In [4]:
cell_cycle_genes = [x.strip() for x in open('./data/cell_cycle_vignette_files/regev_lab_cell_cycle_genes.txt')]
cell_cycle_genes

['MCM5',
 'PCNA',
 'TYMS',
 'FEN1',
 'MCM2',
 'MCM4',
 'RRM1',
 'UNG',
 'GINS2',
 'MCM6',
 'CDCA7',
 'DTL',
 'PRIM1',
 'UHRF1',
 'MLF1IP',
 'HELLS',
 'RFC2',
 'RPA2',
 'NASP',
 'RAD51AP1',
 'GMNN',
 'WDR76',
 'SLBP',
 'CCNE2',
 'UBR7',
 'POLD3',
 'MSH2',
 'ATAD2',
 'RAD51',
 'RRM2',
 'CDC45',
 'CDC6',
 'EXO1',
 'TIPIN',
 'DSCC1',
 'BLM',
 'CASP8AP2',
 'USP1',
 'CLSPN',
 'POLA1',
 'CHAF1B',
 'BRIP1',
 'E2F8',
 'HMGB2',
 'CDK1',
 'NUSAP1',
 'UBE2C',
 'BIRC5',
 'TPX2',
 'TOP2A',
 'NDC80',
 'CKS2',
 'NUF2',
 'CKS1B',
 'MKI67',
 'TMPO',
 'CENPF',
 'TACC3',
 'FAM64A',
 'SMC4',
 'CCNB2',
 'CKAP2L',
 'CKAP2',
 'AURKB',
 'BUB1',
 'KIF11',
 'ANP32E',
 'TUBB4B',
 'GTSE1',
 'KIF20B',
 'HJURP',
 'CDCA3',
 'HN1',
 'CDC20',
 'TTK',
 'CDC25C',
 'KIF2C',
 'RANGAP1',
 'NCAPD2',
 'DLGAP5',
 'CDCA2',
 'CDCA8',
 'ECT2',
 'KIF23',
 'HMMR',
 'AURKA',
 'PSRC1',
 'ANLN',
 'LBR',
 'CKAP5',
 'CENPE',
 'CTCF',
 'NEK2',
 'G2E3',
 'GAS2L3',
 'CBX5',
 'CENPA']

In [5]:
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]

In [6]:
sc.pp.normalize_total(adata, target_sum=1e6)

In [7]:
sc.pp.log1p(adata)
sc.pp.scale(adata)

In [8]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)


/Users/liupeiwen/python/anaconda3/lib/python3.8/site-packages/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):
/Users/liupeiwen/python/anaconda3/lib/python3.8/site-packages/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):


In [10]:
adata.obs[['phase']].to_csv('./data/cellcycle_phase.csv',index=True,header=True)

In [23]:
for i in np.unique(adata.obs['Sub_Cluster']):
    ann = pd.read_csv('./result/cluster/'+i+'_cluster.csv',
                  index_col=0,header=0,sep=',',
                 usecols=['cell','cluster'])
    data = adata[adata.obs['Sub_Cluster']==i]
    data.obs = data.obs.join(ann)
    response = data.obs[['cluster','phase']]
    response.to_csv('./data/Cellcycle_count/'+i+'_cellcycle_count.csv',header=True,index=True)

In [8]:
adata.obs['item'] = adata.obs.index

In [9]:
adata.obs.index = adata.obs['item']

In [10]:
adata.obs.rename(columns={'phase':'label'},inplace=True)
np.unique(adata.obs['label'])

array(['G1', 'G2M', 'S'], dtype=object)

In [11]:
for i in np.unique(adata.obs['Sub_Cluster']) :
    response = adata[adata.obs['Sub_Cluster']==i].obs[['label']]
    response.to_csv('./data/'+i+'_cycle.csv',header=True, index=True, sep=',')